In [1]:
from gensim.summarization.summarizer import summarize

In [2]:
import pandas as pd

df = pd.read_csv("아모레크롤링_스킨케어_완료.csv")

In [69]:
from konlpy.tag import Kkma, Okt
import re

kkma = Kkma()
okt = Okt()

# spacing 이 아예 임포트되지 않아 비슷한 기능이라도 따라했습니다.
def okt_spacing(sentence):
    tagged = okt.pos(sentence)
    corrected = ""
    for word, tag in tagged:
        if tag in ['Noun', 'Adjective', 'Verb']:
            corrected += " " + word
        else:
            corrected += word
    return corrected.strip()



In [74]:
def preprocessing(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        for sentence in kkma.sentences(r):
            sentence = re.sub('([a-zA-Z])','',sentence)
            sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
            sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
            if len(sentence) == 0:
                continue
            if len(sentence) < 198:
                sentence = okt_spacing(sentence)
            sentence += '. '
            total_review += sentence
    return total_review

In [87]:
def is_spam_text(text, max_repeated_chars=10, min_unique_chars=5):
    # 연속된 문자 검사
    for char in set(text):
        if char * max_repeated_chars in text:
            return True
    
    # 고유한 문자 수 검사
    if len(set(re.sub(r'\s+', '', text))) < min_unique_chars:
        return True
    
    return False

In [95]:
def summarize_individual_reviews(product_name, review_list, min_length=30):
    review_summary_dict = {}
    for i, review in enumerate(review_list):
        if is_spam_text(review):
            review_summary_dict[f"{product_name}의 {i+1}번째 리뷰"] = "스팸 또는 부적절한 리뷰로 판단됩니다."
            continue

        preprocessed_review = preprocessing([review])
        
        if len(preprocessed_review.split('.')) < 5 or len(preprocessed_review) < min_length:
            review_summary_dict[f"{product_name}의 {i+1}번째 리뷰"] = review
            continue
            
        try:
            summary = summarize(preprocessed_review, word_count=20)
            summary = re.sub('\n', ' ', summary)
            
            if len(summary) == 0:
                review_summary_dict[f"{product_name}의 {i+1}번째 리뷰"] = review
            else:
                review_summary_dict[f"{product_name}의 {i+1}번째 리뷰"] = summary
        except ValueError:
            review_summary_dict[f"{product_name}의 {i+1}번째 리뷰"] = "요약 중 오류가 발생했습니다."
                
    return review_summary_dict

In [96]:
texts = df[df['상품명'] == "화산송이 모공 바하 클렌징 폼 150g"]['리뷰'].to_list()
result = summarize_individual_reviews("화산송이 모공 바하 클렌징 폼 150g", texts)
result

{'화산송이 모공 바하 클렌징 폼 150g의 1번째 리뷰': '남편이극 지성이에요. 원래 수퍼 화산 송이 미 셀라만 꾸준히 사용 해오 다가 지성 전용이라길래 각지 제거 성분도 있다길래 바꿔 봤어요.',
 '화산송이 모공 바하 클렌징 폼 150g의 2번째 리뷰': '여름이라 집에 와서 메이크업 클렌징 해줄때 꼭 이중세안 해주는데 이 폼 클렌징으로 세안 해주면 좋은점이 뾰루지가 안나고 뽀드득하게 씻겨줘서 좋아요. 세안 브러쉬에 적당하게 손마디 하나로 짜준다음 물을 묻히고 피부결 따라 거품으로 구석구석 닦아주면 좋답니다.',
 '화산송이 모공 바하 클렌징 폼 150g의 3번째 리뷰': '이니 스프 리 폼 클렌징은 피부에 자극적이지 않아 좋아요. 선 크림만 사용 해도 꼭 폼까지 써서 세안 하는데 세안 후에도 당기는 느낌이많이 없어서 만족스럽네요.',
 '화산송이 모공 바하 클렌징 폼 150g의 4번째 리뷰': '저희 엄마가 이 제품이 클렌징이 제일 잘된다고 좋아하서 1+1 이벤트하길래 4개 쟁였어요. 조금만 써도 거품도 잘나고 씻고 나서 남는 느낌도 없으면서 많이 건조하지도 않아서 좋아요. 전에 쓰던거랑 케이스가 달라졌던데 내용물은 똑같았으면 좋겠네요.',
 '화산송이 모공 바하 클렌징 폼 150g의 5번째 리뷰': '피부 트러블도 일어나지 않아서 더 맘에 들어요. 피부가 완전 촉촉해서 맘에 들어요.',
 '화산송이 모공 바하 클렌징 폼 150g의 6번째 리뷰': '이니스프리 화산송이 모공 바하 클렌징폼을 구입해봤어요 이니스프리 화산송이 제품이 유명해서 클렌징폼 말고 다른 팩을 사용해봤었는데 꽤 만족해서 이번에는 요제품을 구입해봤는데 피부에 자극이 없는건 아니지만 격일로 쓰기에 좋은거같아요 겨울보다는 여름에 유용할 것 같습니다',
 '화산송이 모공 바하 클렌징 폼 150g의 7번째 리뷰': '기한25년까지라 유효기간 넉넉하고  제품용량은 150g이에요  보통대용량을 사는데  클렌징폼은 한번쓸때 소량만 쓰기때문에  딱좋다고생각해요  거품도 잘 나구요~  피지과도할때 딥

In [ ]:
def summarize_all_reviews(product_name, review_list, min_length=30, word_count=20):
    all_reviews = ''
    for review in review_list:
        if is_spam_text(review): 
            continue
        preprocessed_review = preprocessing([review])
        all_reviews += preprocessed_review + ' '

    if len(all_reviews) < min_length:
        return {f"{product_name}의 전체 리뷰 요약": all_reviews.strip()}
    

    try:
        summary = summarize(all_reviews, word_count=word_count)
        summary = re.sub('\n', ' ', summary)
        if len(summary) == 0:
            return {f"{product_name}의 전체 리뷰 요약": all_reviews.strip()}
        else:
            return {f"{product_name}의 전체 리뷰 요약": summary}
    except ValueError:
        return {f"{product_name}의 전체 리뷰 요약": "요약 중 오류가 발생했습니다."}


texts = df[df['상품명'] == "화산송이 모공 바하 클렌징 폼 150g"]['리뷰'].to_list()
summary_dict = summarize_all_reviews("화산송이 모공 바하 클렌징 폼 150g", texts)


In [ ]:
summary_dict